# Sistemas Lineares

## Eliminação Gaussiana

In [8]:
A = [3.0 1 2; -1 2 1; 1 1 4]

3x3 Array{Float64,2}:
  3.0  1.0  2.0
 -1.0  2.0  1.0
  1.0  1.0  4.0

In [9]:
b = [6.0; 2; 6]

3-element Array{Float64,1}:
 6.0
 2.0
 6.0

In [4]:
m21 = A[2,1]/A[1,1]

-0.3333333333333333

In [10]:
# Li = A[i,:]
A[2,:] = A[2,:] - m21*A[1,:]

1x3 Array{Float64,2}:
 0.0  2.33333  1.66667

Note que A[2,1] é zero agora

In [11]:
A

3x3 Array{Float64,2}:
 3.0  1.0      2.0    
 0.0  2.33333  1.66667
 1.0  1.0      4.0    

In [12]:
b[2] = b[2] - m21*b[1]

4.0

In [13]:
b

3-element Array{Float64,1}:
 6.0
 4.0
 6.0

In [14]:
m31 = A[3,1]/A[1,1]
A[3,:] = A[3,:] - m31*A[1,:]

1x3 Array{Float64,2}:
 0.0  0.666667  3.33333

In [15]:
b[3] = b[3] - m31*b[1]

4.0

In [16]:
A

3x3 Array{Float64,2}:
 3.0  1.0       2.0    
 0.0  2.33333   1.66667
 0.0  0.666667  3.33333

In [17]:
m32 = A[3,2]/A[2,2]

0.28571428571428575

In [18]:
A[3,:] = A[3,:] - m32*A[2,:]

1x3 Array{Float64,2}:
 0.0  -1.11022e-16  2.85714

In [19]:
b[3] = b[3] - m32*b[2]

2.8571428571428568

In [20]:
A

3x3 Array{Float64,2}:
 3.0   1.0          2.0    
 0.0   2.33333      1.66667
 0.0  -1.11022e-16  2.85714

In [21]:
b

3-element Array{Float64,1}:
 6.0    
 4.0    
 2.85714

## Pontos importantes

- $A$ foi perdida. Ela é irrecuperável (sem os valores de $m_{i,j}$.
    - Sol. 1: Não "usar" $A$, i.e., copiar para outra matriz
    - Sol. 2: Guardar $m_{i,j}$.
    - Sol. 3: Ambos
- A eliminação do elemento $a_{i,j}$ faz contas desnecessárias com 0.0 (antes da coluna $j$).
- $m_{i,j}$ foi calculado com o objetivo de transformar
$a_{i,j}$ em 0.0. Então não é necessário calcular a atualização de $a_{i,j}$. (Basta fazer `A[i,j] = 0.0`).
Sem falar que o resultado pode não ser exatamente 0.0, e
ficar sujando as contas.
- A matriz resultante tem muitos zeros, i.e., gasto de memória a toa.
    - Sol. : Usar esse espaço vazio.

In [129]:
A = [3.0 1 2; -1 2 1; 1 1 4]
U = copy(A)
b = [6.0;2;6]
c = copy(b)

3-element Array{Float64,1}:
 6.0
 2.0
 6.0

In [130]:
# Coluna 1
m21 = U[2,1]/U[1,1]
m31 = U[3,1]/U[1,1]
U[2,1] = 0.0
U[3,1] = 0.0
U[2,2:end] = U[2,2:end] - m21*U[1,2:end]
U[3,2:end] = U[3,2:end] - m31*U[1,2:end]
c[2] = c[2] - m21*c[1]
c[3] = c[3] - m31*c[1]
U,c

(
3x3 Array{Float64,2}:
 3.0  1.0       2.0    
 0.0  2.33333   1.66667
 0.0  0.666667  3.33333,

[6.0,4.0,4.0])

In [131]:
# Coluna 2
m32 = U[3,2]/U[2,2]
U[3,2] = 0.0
U[3,3:end] = U[3,3:end] - m32*U[2,3:end]
c[3] = c[3] - m32*c[2]
U,c

(
3x3 Array{Float64,2}:
 3.0  1.0      2.0    
 0.0  2.33333  1.66667
 0.0  0.0      2.85714,

[6.0,2.0,6.0])

Computacionalmente, temos mais um problema.

- `A[i,j+1:end]` cria um vetor temporario, então
$L_i \leftarrow L_i - m_{i,j}L_j$ tem um gasto de memória desnecessário.
    - Nesta disciplina, vamos ignorar esse fato, sempre que não afetar demais.
- A maneira correta de se calcular
$L_i \leftarrow L_i - m_{i,j}L_j$ é com `for`.

In [132]:
function elim_gauss(A, b)
    m,n = size(A)
    U = copy(A)
    c = copy(b)
    @inbounds for j = 1:n
        @inbounds for i = j+1:m
            mij = U[i,j]/U[j,j]
            U[i,j] = 0.0
            @inbounds for k = j+1:n
                U[i,k] -= mij*U[j,k]
            end
            c[i] -= mij*c[j]
        end
    end
    return U, c
end

elim_gauss (generic function with 2 methods)

In [118]:
A

1000x1000 Array{Float64,2}:
 1000.93          0.693143      0.0130765  …     0.209335       0.398884 
    0.752804   1000.22          0.380064         0.204649       0.658994 
    0.554684      0.915637   1000.36             0.920568       0.515883 
    0.238366      0.178041      0.16035          0.162372       0.75579  
    0.438306      0.394185      0.599904         0.117229       0.887862 
    0.241353      0.222437      0.486493   …     0.0942967      0.327892 
    0.130382      0.0137715     0.190705         0.765321       0.433069 
    0.781277      0.0456453     0.0681019        0.00770053     0.761459 
    0.588024      0.879935      0.0385946        0.153698       0.171976 
    0.920086      0.337919      0.370471         0.63461        0.530219 
    0.486398      0.563834      0.52112    …     0.564202       0.431951 
    0.227429      0.339987      0.249071         0.590638       0.272102 
    0.221188      0.228409      0.622186         0.509937       0.89724  
    ⋮     

In [133]:
U,c = elim_gauss(A,b)

(
3x3 Array{Float64,2}:
 3.0  1.0      2.0    
 0.0  2.33333  1.66667
 0.0  0.0      2.85714,

[6.0,4.0,2.8571428571428568])

In [134]:
function elim_gauss_resumida(A,b)
    m,n = size(A)
    U = copy(A)
    c = copy(b)
    @inbounds for j = 1:n
        mj = U[j+1:m,j]/U[j,j]
        U[j+1:m,j] = 0.0
        c[j+1:m] -= mj*c[j]
        U[j+1:m,j+1:n] -= mj*U[j,j+1:n]
    end
    return U
end

elim_gauss_resumida (generic function with 2 methods)

In [135]:
norm(U - elim_gauss_resumida(A)) < 1e-12

true

In [139]:
# Gerar uma matriz especifica que
# eu sei que funcione
n = 1000
A = rand(n,n) + n*eye(n);
b = A*ones(n);

In [140]:
@time U, c = elim_gauss(A, b);
@time U, c = elim_gauss(A, b);

  0.710672 seconds (10 allocations: 7.637 MB, 0.16% gc time)
  2.401199 seconds (10 allocations: 7.637 MB)


In [141]:
@time U, c = elim_gauss_resumida(A, b);
@time U, c = elim_gauss_resumida(A, b);

  2.235418 seconds (70.23 k allocations: 7.472 GB, 9.93% gc time)
  2.058017 seconds (38.71 k allocations: 7.471 GB, 8.57% gc time)


A alocação não acontece toda de uma vez.
O `gc` (garbage collector) que lida com isso.
Note que no segundo caso, o gc roda muito mais.

## Complexidade

Vamos contar o número de operações elementares

    0 function elim_gauss(A, b)
    0     m,n = size(A)
    0     U = copy(A)
    0     c = copy(b)
    x     @inbounds for j = 1:n
    x         @inbounds for i = j+1:m
    1             mij = U[i,j]/U[j,j]
    0             U[i,j] = 0.0
    x             @inbounds for k = j+1:n
    2                 U[i,k] -= mij*U[j,k]
    0             end
    2             c[i] -= mij*c[j]
    0         end
    0     end
    0     return U, c
    0  end

No for mais interno temos 2 operações que são feitas $n-j$ vezes. Então resumindo

    x     for j = 1:n
    x         for i = j+1:m
    1             mij = U[i,j]/U[j,j]
    0             U[i,j] = 0.0
    2(n-j)        for k = j+1:n ... end
    2             c[i] -= mij*c[j]
    0         end
    0     end

O segundo for não varia o $j$ também, então podemos somar tudo e multiplicar por $m-j$.

    x             for j = 1:n
    (3+2(n-j))*(m-j)  for i = j+1:m ... end
    0             end

A última quantidade depende de $j$, e fica algo do tipo
$$ \sum_{j = 1}^n (3+2(n-j))(m-j) $$

Note que, para isso acontecer, devemos ter $m \geq n$. Caso contrário o segundo for acaba antes, e a complexidade muda um pouco.

O resultado disso é
$$ n^2m - \frac{n^3}{3} - \frac{3n^2}{2} + 2mn - \frac{7n}{6} $$
que quando $m = n$ vira
$$ \frac{2n^3}{3} + \frac{n^2}{2} - \frac{7n}{6}, $$
que é $\mathcal{O}(n^3)$.

Note que a transformação de $b$ em $c$ também está embutida nesse cálculo.